In [20]:
import json
import cv2
import os
from scipy import misc
import shutil
from matplotlib import pyplot as plt
import numpy as np
import numpy.ma as ma
from random import shuffle
%matplotlib inline

In [21]:
animal = 'koala'
raw_data_dir = os.path.join('doodles', animal)

In [22]:
conf_path = os.path.join(raw_data_dir, 'conf.json')
with open(conf_path) as f:
    conf = json.loads(f.read())
obj_num = conf['obj_num']
match = conf['match']

In [23]:
img_names = os.listdir(raw_data_dir)
records = []
for img_name in img_names:
    # only fetch the original image
    if "mask" in img_name or "json" in img_name or "result" in img_name:
        continue
    
    # get the image
    img_path = os.path.join(raw_data_dir, img_name)
    img = misc.imread(img_path)
    
    # verify bone json file exist
    bone_name = img_name.split('.')[0] + ".json"
    bone_path = os.path.join(raw_data_dir, bone_name)
    if os.path.isfile(bone_path) == False:
        continue
    
    # get the bounding boxes
    bboxes = []
    for label_idx in range(obj_num):
        mask_path = img_path.replace("png", "mask." + str(label_idx) + ".png")
        cur_label_img = misc.imread(mask_path, mode='L')
        cur_label_img = np.minimum(cur_label_img, np.ones(cur_label_img.shape))
        
        t = np.nonzero(cur_label_img)
        ymin = np.min(t[0])
        ymax = np.max(t[0])
        xmin = np.min(t[1])
        xmax = np.max(t[1])
        
#         cv2.rectangle(img,
#                       (int(xmin), int(ymin)),
#                       (int(xmax), int(ymax)),
#                       (255, 0, 0),
#                       3)
        
#         plt.imshow(img)
        bboxes.append([xmin, ymin, xmax, ymax])


    # get the bone points
    with open(bone_path) as f:
        bone_info = json.load(f)

    # normalize each bone point by the segmented box
    norm_bone = []
    for bone_idx, bone in enumerate(bone_info):
        box_idx = match[bone_idx]
        bbox = bboxes[box_idx]
        [xmin, ymin, xmax, ymax] = bbox
        xcenter = (xmax + xmin) / 2
        ycenter = (ymax + ymin) / 2
        box_width = (xmax - xmin)
        box_height = (ymax - ymin)
        
        norm_bone_x = (bone[0] - xcenter) / box_width
        norm_bone_y = (bone[1] - ycenter) / box_height
        norm_bone.append([norm_bone_x, norm_bone_y])
    
    norm_bone = [str(ele[0]) + " " + str(ele[1]) for ele in norm_bone]
    norm_bone = ' '.join(norm_bone)
    
    record = img_path + " " + norm_bone + "\n"
    
    records.append(record)

shuffle(records)
print("Number of records: " + str(len(records)))

Number of records: 52


In [24]:
# split into training set and test set
test_ratio = 0.1
total_num = len(records)
test_num = int(test_ratio * total_num)
train_num = total_num - test_num
train_records = records[0:train_num]
test_records = records[train_num:]

In [25]:
# save to text file
all_out_file = open(animal + "_all.txt", "w")
for record in records:
    all_out_file.write(record)
all_out_file.close()

train_out_file = open(animal + "_train.txt", "w")
for record in train_records:
    train_out_file.write(record)
train_out_file.close()

test_out_file = open(animal + "_test.txt", "w")
for record in test_records:
    test_out_file.write(record)
test_out_file.close()